# SD202 TP3 - Views, Updates and Database Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a marketplace


For the first 2 parts we will use the _wine_ database and the Tables created in TP2.

A reminder of the wine database schema:

+-----------------------------------------+
|                 Master1                 |
+-----------------------------------------+
| NV:      Wine number                    |
| CRU:     Vineyard or group of vineyards |
| DEGRE:   Alcohol content                |
| MILL:    Vintage year                   |
| QTE:     Number of bottles harvested    |
| NP:      Producer number                |
| NOM:     Producer's last name           |
| PRENOM:  Producer's first name          |
| REGION:  Wine growing region            |
+-----------------------------------------+

+-------------------------------------------------------------------+
|                              Master2                              |
+-------------------------------------------------------------------+
| NV:      Wine number                                              |
| CRU:     Vineyard or group of vineyards                           |
| DEGRE:   Alcohol content                                          |
| MILL:    Vintage year                                             |
| DATES:   Buying date                                              |
| LIEU:    Place where the wine was sold                            |
| QTE:     Number of bottles bought                                 |
| NB:      Client (buveur) number                                   |
| NON:     Client's last name                                       |
| PRENOM:  Client's first name                                      |
| TYPE:    Type of client by volume of purchases                    |
| REGION:  Administrative Region (different to wine growing region) |
+-------------------------------------------------------------------+

In [1]:
import sqlite3

In [2]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
Buying:
  0: NV(NUM) *1
  1: NB(NUM) *2
  2: DATES(TEXT)
  3: LIEU(TEXT)
  4: QTE(NUM)
Clients:
  0: NB(NUM) *1
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
Lieux:
  0: LIEU(TEXT) *1
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
RBB:
  0: NB(NUM) *1
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
Wines:
  0: NV(NUM) *1
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


Again, we recommend inline %sql as an alternative to the sqlite3 package

In [4]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

Recreate the Tables in 3NF/BCNF from Master2 as you did in the TP2

In [5]:
# NV -> CRU, DEGREE, MILL
%sql DROP TABLE IF EXISTS Wines;
%sql CREATE TABLE Wines (NV NUM, CRU TEXT, DEGRE NUM, MILL NUM, PRIMARY KEY (NV));
%sql INSERT INTO Wines SELECT DISTINCT NV, CRU, DEGRE, MILL FROM Master2 WHERE NV NOT NULL;

# NB -> NOM, PRENOM, TYPE

%sql DROP TABLE IF EXISTS Clients;
%sql CREATE TABLE Clients (NB NUM, NOM TEXT, PRENOM TEXT, TYPE TEXT, PRIMARY KEY (NB));
%sql INSERT INTO Clients SELECT DISTINCT NB, NOM, PRENOM, TYPE FROM Master2 WHERE NB NOT NULL;

# LIEU -> REGION
%sql DROP TABLE IF EXISTS Lieux;
%sql CREATE TABLE Lieux (LIEU TEXT, REGION TEXT, PRIMARY KEY (LIEU));
%sql INSERT INTO Lieux SELECT DISTINCT LIEU, REGION FROM Master2 WHERE LIEU NOT NULL;

# NV, NB -> DATES, LIEU, QTE
%sql DROP TABLE IF EXISTS Buying;
%sql CREATE TABLE Buying (NV NUM, NB NUM, DATES TEXT, LIEU TEXT, QTE NUM, PRIMARY KEY (NV, NB));
%sql INSERT INTO Buying SELECT DISTINCT NV, NB, DATES, LIEU, QTE FROM Master2 WHERE LIEU NOT NULL;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
102 rows affected.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
100 rows affected.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
18 rows affected.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
73 rows affected.


[]

___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```sql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command is:

```sql
DROP VIEW IF EXISTS view_name;
```


__Note:__ Use it with caution (only drop something if you are sure)

__1.1__ Create a view 'bons_buveurs' with the clients (buveurs) of type 'gros' or 'moyen'.

In [6]:
%sql DROP VIEW IF EXISTS bons_buveurs;
%sql CREATE VIEW bons_buveurs AS SELECT NB, NOM, PRENOM, TYPE FROM Clients\
WHERE TYPE='moyen' OR TYPE='gros';

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.


[]

In [7]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


__1.2__ Create the view 'buveurs_asec' with clients (buveurs) who have not bought any wine.

In [8]:
%sql DROP VIEW IF EXISTS buveurs_asec;
%sql CREATE VIEW buveurs_asec AS SELECT NB, NOM, PRENOM, TYPE FROM Clients\
WHERE NB NOT IN (SELECT NB FROM Buying WHERE QTE NOT NULL AND QTE!=0);

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.


[]

In [9]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


__1.3__ Create the view 'buveurs_achats' complementary to the previous one.

In [10]:
%sql DROP VIEW IF EXISTS buveurs_achats;
%sql CREATE VIEW buveurs_achats AS SELECT NB, NOM, PRENOM, TYPE FROM Clients\
WHERE NB IN (SELECT NB FROM Buying WHERE QTE NOT NULL AND QTE!=0);

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.


[]

In [11]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,petit
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
9,Ajar,Emile,petit
10,Andersen,Yann,gros


__1.4__ Create the view 'q83pl' (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [12]:
%sql DROP VIEW IF EXISTS q83pl;
%sql CREATE VIEW q83pl AS SELECT LIEU, CRU, SUM(QTE) AS QTE_BUE FROM Buying, Wines\
WHERE Buying.NV=Wines.NV AND DATES LIKE '1983-%' GROUP BY LIEU, CRU;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.


[]

In [13]:
# Test
%sql SELECT * FROM q83pl;

 * sqlite:///wine.db
Done.


LIEU,CRU,QTE_BUE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


__1.5__ Can we define the same view with ascending order over the attribute QTE? Provide an explanation for your answer.

In [14]:
%sql SELECT * FROM q83pl ORDER BY QTE_BUE ASC;

 * sqlite:///wine.db
Done.


LIEU,CRU,QTE_BUE
PARIS,Coteaux du Tricastin,1
RENNES,Mercurey,1
LYON,Julienas,2
CAEN,Seyssel,3
PARIS,Pouilly Vinzelles,3
PARIS,Beaujolais Primeur,4
LILLE,Pommard,5
LYON,Beaujolais Villages,10
ROCQUENCOURT,Saint Amour,80
ROCQUENCOURT,Beaujolais Villages,260


___
# PART II: UPDATE DATABASE CONTENT

The syntax for the Update statement is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the Insert statement is:

```sql
INSERT INTO TABLE_NAME [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```


Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you may not execute a DELETE, INSERT or UPDATE statement on a view.

__2.1__ Create a table 'RBB' with the same schema as 'bons_buveurs' which contains the tuples selected from 'bons_buveurs'

In [15]:
%sql DROP TABLE IF EXISTS RBB;
%sql CREATE TABLE RBB (NB NUM, NOM TEXT, PRENOM TEXT, TYPE TEXT, PRIMARY KEY (NB));
%sql INSERT INTO RBB SELECT * FROM bons_buveurs;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
56 rows affected.


[]

In [16]:
# Test
%sql SELECT * FROM RBB ORDER BY NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


__2.2__ Update the table you used to create 'bons_buveurs': Change the 'type' to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may be different from the one in your table)

In [17]:
%sql SELECT Clients.NB, NOM, PRENOM, TYPE, SUM(QTE) AS TOTAL FROM Clients, Buying\
WHERE Buying.NB=Clients.NB GROUP BY Clients.NB HAVING TOTAL > 100 AND TYPE!='gros';

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,TOTAL
2,Artaud,Antonin,moyen,583
5,Audiberti,Jacques,petit,113
9,Ajar,Emile,petit,140
44,Gide,Andre,petit,171


Update instances

In [18]:
%sql UPDATE Clients SET TYPE='gros'\
WHERE NB IN(SELECT NB FROM Buying GROUP BY NB HAVING SUM(QTE) > 100);

 * sqlite:///wine.db
5 rows affected.


[]

__2.3__ Compare the content of _table_ 'RBB' and the _view_ 'bons_buveurs' after the update. What differences do you see? Explain

In [19]:
%sql SELECT RBB.NB, NOM, PRENOM, TYPE, SUM(QTE) AS TOTAL FROM RBB, Buying\
WHERE RBB.NB=Buying.NB GROUP BY RBB.NB HAVING TOTAL > 100;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,TOTAL
2,Artaud,Antonin,moyen,583
8,Aragon,Louis,gros,132


In [20]:
%sql SELECT bons_buveurs.NB, NOM, PRENOM, TYPE, SUM(QTE) AS TOTAL FROM bons_buveurs, Buying\
WHERE bons_buveurs.NB=Buying.NB GROUP BY bons_buveurs.NB HAVING TOTAL > 100;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,TOTAL
2,Artaud,Antonin,gros,583
5,Audiberti,Jacques,gros,113
8,Aragon,Louis,gros,132
9,Ajar,Emile,gros,140
44,Gide,Andre,gros,171


__2.4__ Create a table 'RBA' with the same schema as 'buveurs_asec' which contains the tuples selected from 'buveurs_asec'

In [21]:
%sql DROP TABLE IF EXISTS RBA;
%sql CREATE TABLE RBA (NB NUM, NOM TEXT, PRENOM TEXT, TYPE TEXT, PRIMARY KEY (NB));
%sql INSERT INTO RBA SELECT * FROM buveurs_asec;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
57 rows affected.


[]

In [22]:
# Test
%sql SELECT * FROM RBA;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


__2.5__ Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to 'RBA'

In [23]:
%sql INSERT INTO RBA VALUES(101, 'Karst', 'Theo', 'petit');

 * sqlite:///wine.db
1 rows affected.


[]

In [24]:
# Test
%sql SELECT * FROM RBA;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


__2.6__ Compare the content of _table_ 'RBA' and the _view_ 'buveurs_asec'. What differences do you see? Explain

In [25]:
%sql SELECT * FROM buveurs_asec WHERE NB=101;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE


In [26]:
%sql SELECT * FROM RBA WHERE NB=101;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
101,Karst,Theo,petit


We can see that the view buveurs_asec wasn't modified when we added a row in the table RBA. This was expected, since buveurs_asec is a view of the table Clients, and thus only depends on this table. RBA is a copy of buveurs_asec, saved in a different table. Thus, modifying RBA won't change buveurs_asec.

___
# PART III: Design the database schema for a marketplace

We consider the following scenario:

You are a developer for a game. In the game, there are several kinds of objects that the players can collect (e.g. magical flowers, gems, etc.). Players can also sell or buy objects from a marketplace that works in the following way: players declare some of the objects in their possession as “buyable”
with a price for each buyable object and then any player can buy items marked as buyable. When a player buys an item, the possession of the item is transferred to the buyer while the money corresponding to the item is transferred from the buyer to the seller. Note that, as long as the item is not sold, it still belongs to the player. Your goal is to
create a database for this marketplace.

The database schema you will develop needs to support players, the money each player has, the various types
of objects present in the game and how many objects of each kind each player has.Your database need to be able
to support the following kind of operations:

* Create a new type of object or add new players,
* Change the name of a type of objects or the name of a player,
* Attribute an object of a given type to a player,
* Increase or decrease the amount of money a player has,
* Retrieve the list of all the items that a player has,
* Compute the current balance of a player,
* Allow a player to mark one of their item as buyable with a given price,
* Allow a player to buy the cheapest item of a given type from the marketplace.

Ideally your schema should enforce the following constraints:

* All the objects that the players have correspond to some known type,
* Each object is possessed by exactly one player,
* Any single object cannot be marked twice as buyable, nor it can have two different prices (but two different objects of the same kind can have different prices),
* Any player has a positive amount of money.

__3.1__ Draw an entity-relationship model for the scenario.

_to answer, you can for example join a figure with the notebook or write the information related to the entity and relationship sets_

For this scenario, we can use the following entity-relationship model:

+------------------------------------------------------+
|                      ObjectTypes                     |
+------------------------------------------------------+
| id:        ID of the type                            |
| name:      Name of the type (magical flower, etc...) |
+------------------------------------------------------+
                            |  (0, N)
                            |
                            |
                            |  (1, 1)
+------------------------------------------------------+
|                        Objects                       |
+------------------------------------------------------+
| id:        ID of the object                          |
| type:      ID of the object type                     |
| player:    ID of the player having this object       |
| buyable:   If the object is for sale                 |
| price:     The price of the object                   |
+------------------------------------------------------+
                            |  (1, 1)
                            |
                            |
                            |  (0, N)
+------------------------------------------------------+
|                        Players                       |
+------------------------------------------------------+
| id:        ID of the player                          |
| name:      Name of the player                        |
| balance:   Money of the player                       |
+------------------------------------------------------+

- ObjectTypes: Contains the list of all the possible types for objects (magical flower, etc...)
- Objects: Contains all the objects in the game
- Players: Contains all the players in the game

We know that every relationship associated to an entity class with cardinality (0,1) or (1,1) can be merged with the entity class.
Since the entity Objects has cardinalities of (1,1), we don't need to use relationships for this model because they can be merged with the Object entity class.

__3.2__ Deduce the relational scheme while precising the _primary_ and _foreign_ (if any) keys for each relation.

In [ ]:
%sql CREATE TABLE ObjectTypes(
    id INT Primary Key, 
    name VARCHAR
    );

%sql CREATE TABLE Players(
    id INT Primary Key,
    name VARCHAR,
    balance INT
);

%sql CREATE TABLE Objects(
    id INT Primary Key,
    type INT FOREIGN KEY REFERENCES ObjectTypes(id),
    player INT FOREIGN KEY REFERENCES Players(id),
    buyable BOOLEAN,
    price INT);